In [1]:
!pip install transformers datasets sentencepiece torch accelerate peft bitsandbytes

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 289.2 kB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.5/40.5 kB 369.9 kB/s eta 0:00:00 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.7/57.7 kB 48.5 kB/s eta 0:00:00 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.2/69.2 kB 710.4 kB/s eta 0:00:00 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 5.5 MB/s eta 0:00:00:00:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.4/485.4 kB 5.3 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 5.2 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 342.1/342.1 kB 3.8 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 374.8/374.8 kB 3.5 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 866.2 kB/s eta 0:00:000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 7.6 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 

In [2]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, BitsAndBytesConfig
from peft import get_peft_model, LoraConfig, TaskType
import torch
import pandas as pd
from datasets import Dataset
import numpy as np


НАСТРОЙКА МОДЕЛИ

In [3]:
# Название модели
MODEL_NAME = "ai-forever/ruT5-large"

# Определяем устройство (GPU/CPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Конфигурация 8-битного квантования
quantization_config = BitsAndBytesConfig(
    load_in_8bit=True  # Снижаем использование памяти
)

# Загружаем токенизатор
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, legacy=False)

# Загружаем модель с 8-битным квантованием
model = AutoModelForSeq2SeqLM.from_pretrained(
    MODEL_NAME,
    quantization_config=quantization_config,
    device_map="auto"
)

# Отключаем кеширование, так как оно конфликтует с gradient checkpointing
model.config.use_cache = False


tokenizer_config.json:   0%|          | 0.00/20.4k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/1.00M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.37k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/2.95G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.95G [00:00<?, ?B/s]

Настройка LoRA

In [4]:
# Конфигурация LoRA
lora_config = LoraConfig(
    task_type=TaskType.SEQ_2_SEQ_LM,
    r=8,
    lora_alpha=32,
    lora_dropout=0.1,
)

# Применяем LoRA к модели
model = get_peft_model(model, lora_config)

# Включаем градиенты только для LoRA-адаптеров
for param in model.parameters():
    if param.dtype in [torch.float16, torch.float32]:  
        param.requires_grad = True


Загрузка и предобработка данных

In [5]:
# Загружаем CSV-файл с данными
df = pd.read_csv("output_final.csv", sep=";")

# Удаляем строки с пропущенными значениями
df = df.dropna()

# Исправляем возможные ошибки в названиях столбцов
df.rename(columns={"text_wich_errors": "text_with_errors"}, inplace=True)

# Преобразуем Pandas DataFrame в Hugging Face Dataset
dataset = Dataset.from_pandas(df)

# Разделяем данные на train (90%) и validation (10%)
dataset = dataset.train_test_split(test_size=0.1, seed=42)

# Выводим доступные колонки
print("Столбцы в dataset:", dataset["train"].column_names)


Столбцы в dataset: ['text_with_errors', 'corrected_text', '__index_level_0__']


Предобработка текстов

In [6]:
# Добавляем промпт в исходный текст
def preprocess_data(example):
    return {
        "input_text": "Исправь текст: " + example["text_with_errors"],
        "target_text": example["corrected_text"]
    }

# Применяем предобработку к датасету
dataset = dataset.map(preprocess_data)


Map:   0%|          | 0/182026 [00:00<?, ? examples/s]

Map:   0%|          | 0/20226 [00:00<?, ? examples/s]

Токенизация текстов

In [7]:
def tokenize_function(examples):
    model_inputs = tokenizer(
        examples["input_text"],
        padding="max_length",
        truncation=True,
        max_length=512,
        return_tensors="pt"
    )

    labels = tokenizer(
        examples["target_text"],
        padding="max_length",
        truncation=True,
        max_length=512,
        return_tensors="pt"
    )

    model_inputs["labels"] = labels["input_ids"]
    
    return model_inputs

# Токенизируем обучающую и валидационную выборки
tokenized_train_dataset = dataset["train"].map(tokenize_function, batched=True)
tokenized_eval_dataset = dataset["test"].map(tokenize_function, batched=True)

# Удаляем ненужные текстовые колонки
tokenized_train_dataset = tokenized_train_dataset.remove_columns(["input_text", "target_text"])
tokenized_eval_dataset = tokenized_eval_dataset.remove_columns(["input_text", "target_text"])


Map:   0%|          | 0/182026 [00:00<?, ? examples/s]

Map:   0%|          | 0/20226 [00:00<?, ? examples/s]

Настройки обучения

In [8]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="./ruT5-corrector",
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=5e-5,
    per_device_train_batch_size=1,  # Минимальный batch size
    per_device_eval_batch_size=1,
    gradient_accumulation_steps=16,  # Градиентное накопление
    weight_decay=0.01,
    bf16=True,  # Используем `bf16`, т.к. он лучше совместим с 8-битными моделями
    logging_dir="./logs",
    logging_steps=500,
    optim="adamw_bnb_8bit",  # Оптимизатор для 8-битного обучения
    label_names=["labels"], # Явно указываем labels
    num_train_epochs=2,  # Устанавливаем 5 эпох
)


Создание Trainer и запуск обучения

In [ ]:
from transformers import Trainer, DataCollatorForSeq2Seq

# Коллектор данных (автоматически добавляет padding)
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

# Функция вычисления метрик
def compute_metrics(eval_pred):
    preds, labels = eval_pred
    preds = np.where(preds != -100, preds, tokenizer.pad_token_id)
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    return {"accuracy": np.mean([p == l for p, l in zip(decoded_preds, decoded_labels)])}

# Создаем Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_dataset,
    eval_dataset=tokenized_eval_dataset,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

# Запускаем обучение
trainer.train()


/opt/conda/lib/python3.11/site-packages/bitsandbytes/autograd/_functions.py:315: UserWarning: MatMul8bitLt: inputs will be cast from torch.bfloat16 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")


Epoch,Training Loss,Validation Loss
